In [7]:
## Notebook to train the models and store them in the models folder

In [15]:
from Utils import *

In [16]:
#path to models directory: (TODO: change to the teacher's requirement)
models_path="models"

In [17]:
#Load the dataset:
# Create training, validation and test partitions
quora_df = pd.read_csv("./quora_data.csv")
A_df, test_df = sklearn.model_selection.train_test_split(quora_df, test_size=0.05, random_state=123)
train_df, val_df = sklearn.model_selection.train_test_split(A_df, test_size=0.05)
print('train_df.shape=',train_df.shape)

# cast to list taking care of nans:
q1_train =  cast_list_as_strings(list(train_df["question1"]))
q2_train =  cast_list_as_strings(list(train_df["question2"]))

all_questions = q1_train + q2_train

print(f'lenght of all questions: {len(all_questions)}')

train_df.shape= (291897, 6)
lenght of all questions: 583794


## 1. Train and save model of Teacher for Baseline

In [19]:
#train and save the teacher's baseline with count vectorizer:
count_vectorizer = sklearn.feature_extraction.text.CountVectorizer(ngram_range=(1,1))
count_vectorizer.fit(all_questions)

X_tr_q1q2 = get_features_from_df(train_df, count_vectorizer)

logistic = sklearn.linear_model.LogisticRegression(solver="liblinear",
                                                   random_state=123)
y_train = train_df["is_duplicate"].values
logistic.fit(X_tr_q1q2, y_train)

save_model(logistic, models_path, "teacher_baseline.pkl")
save_model(count_vectorizer, models_path, "count_vectorizer.pkl")

Model saved to models/teacher_baseline.pkl
Model saved to models/count_vectorizer.pkl


## 2. Train and save model using custom TFIDF vectorizer:

In [20]:
print('fit custom vectorizer on ALL questions...')
custom_vectorizer = TFIDF_Vectorizer()
custom_vectorizer.fit(all_questions)

print('Transforming the question1 and question2 columns from the train data \
by applying the fitted vectorizer\
and putting them in a sparce matrix...')
X_tr_q1q2_tfidf = get_features_from_df_tfidf(train_df, custom_vectorizer)
y_train = train_df["is_duplicate"].values

# regression similoar to bvaseline to see if tfidf makes things better:
logistic2 = sklearn.linear_model.LogisticRegression(solver="liblinear",
                                                   random_state=123)
logistic2.fit(X_tr_q1q2_tfidf, y_train)

save_model(logistic2, models_path, "custom_tfidf_logreg.pkl")
save_model(custom_vectorizer, models_path, "custom_tfidf_vectorizer.pkl")

fit custom vectorizer on ALL questions...
Transformin the question1 and question2 columns from the train data by applying the fitted vectorizerand putting them in a sparce matrix...
Model saved to models/custom_tfidf_logreg.pkl
Model saved to models/custom_tfidf_vectorizer.pkl
